<h1>Language Model</h1>

In [ ]:
import timeit

query = '1991'
clean_query_text, clean_query_words = preprocess(query)
lambd = 1/4
limit = 5
result_dict = list()

df_doc_freq = pd.read_csv('Document Frequencies.csv')

global_freq = int(open("global_frequency.txt", "r").read())
print(global_freq)

start = timeit.default_timer()
for i in df_doc_freq.iterrows():
    local_freq = i[1]['Frequency']
    doc_number = i[1]['Document']
    doc_number = str(doc_number).zfill(3)
    result = 1
    for j in query.split():
        entries = df.loc[df['Term'] == j]['Documents'].values[0]
        entries = entries.split(', ',1)
        entries[0] = entries[0].replace("[", "")
        entries[1] = entries[1].replace(']', '')
        entries[1] = ast.literal_eval(entries[1])
        if doc_number in entries[1].keys():
            prob = (float(entries[1][doc_number])/float(local_freq)*lambd) + (float(entries[0])/float(global_freq)*(1-lambd))
        else:
            prob = (float(entries[0])/float(global_freq)*(1-lambd))
        result *= prob
    result_dict.append((doc_number, result))
result_sorted = sorted(result_dict, key=lambda x: x[1], reverse=True)
print(result_sorted[:limit])

stop = timeit.default_timer()
print("Waktu pembuatan indeks:", stop-start)

<h1>TF IDF</h1>

In [ ]:
import os
import nltk
import re
import pandas as pd
import string
import ast
from itertools import islice
import math

df_tfidf = pd.read_csv('Konstruksi Indeks.csv')
doc = df_tfidf['Documents']
term = df_tfidf['Term']
df_tfidf
print(df_tfidf)

query = "1991"
data = query.split()
z=1
count = 1
N = 500+1

dict_w = dict()
dict_f = dict()
dict_tf = dict()
dict_idf = dict()
dict_dj = dict()
dict_sim = dict()

for q in data:
    for i in range (0,max_range):
        ni = 0
        start = 2
        end = 5
        count = 1
        if (term[i] == q):
            for y in range (0,int(len(doc[i])/7)):
                if(int(len(doc[i])/7)!=1):
                    if(doc[i][start:end] != doc[i][(start-7):(end-7)]):
                        ni+=1
                        dict_f['f',doc[i][start:end]]=count
                    else:
                        dict_f['f',doc[i][start:end]]=count
                        count+=1
                        ni=1
                else:
                    dict_f['f',doc[i][start:end]]=count
                    ni+=1
                    
                start = start+7
                end = end+7
                
            dict_idf[q] = math.log((1+N/ni),10)

            
for key in dict_f:
    dict_tf['tf',z] = 1+math.log(dict_f[key],10)
    z+=1

z=1

for key in dict_tf:
    for key2 in dict_idf:
        dict_w['w',z] = dict_tf[key]*dict_idf[key2]
        z+=1

z=1

for key in dict_w:
    dict_dj['dj',z] = math.sqrt(pow(dict_w[key],2))
    z+=1

z=1
for key in dict_dj:
    for key2 in dict_w:
        dict_sim["d",z] = dict_w[key2]/dict_dj[key]
        z+=1
        break
        
print(data)
print("N : ",N)
print("f : ",dict_f)
print("Tf : ",dict_tf)
print("IDF : ",dict_idf)
print("W : ", dict_w)
print("Panjang : ", dict_dj)
print("Sim : ",dict_sim)